In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications import EfficientNetB0

import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

import tempfile
from os import path
%load_ext tensorboard

# Global settings

In [3]:
NUM_EPOCHS = 2

### Train and save model 

In [4]:
def train_model(model, train_X, train_y, model_path, batch_size=64):
    if path.exists(model_path):
        print("Model is already trained and saved here: " + model_path)
        return
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    checkpoint_filepath = './tmp/'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    model.fit(
        train_images,
        train_labels,
        epochs=NUM_EPOCHS,
        validation_split=0.1,
        batch_size=batch_size,
        callbacks=[model_checkpoint_callback]
    )
    
    # save model
    model.save(model_path)
    print('Saved to: ' + model_path)

### wrap model

In [5]:
def wrap_model(base_model, num_classes):
    inputs = base_model.inputs
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Image classification models

## CIFAR classification

In [6]:
NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)

In [7]:
def load_cifar_dataset():
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    train_images = tf.cast(train_images, tf.float32)
    test_images = tf.cast(test_images, tf.float32)
    
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    return (train_images, train_labels), (test_images, test_labels)

In [8]:
(train_images, train_labels), (test_images, test_labels) = load_cifar_dataset()

### mobilenetV2

In [9]:
MODEL_PATH = './cifar_models/MobileNetV2_cifar_model.h5'

In [10]:
base_model = MobileNetV2(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./cifar_models/MobileNetV2_cifar_model.h5


### EfficientNets - B0, B4

https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html

#### EfficientNetB0

In [11]:
MODEL_PATH = './cifar_models/EfficentNetB0_cifar_model.h5'

In [12]:
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./cifar_models/EfficentNetB0_cifar_model.h5


#### EfficientNetB4

In [13]:
MODEL_PATH = './cifar_models/EfficentNetB4_cifar_model.h5'

In [14]:
base_model = EfficientNetB4(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./cifar_models/EfficentNetB4_cifar_model.h5


## MNIST dataset

In [15]:
NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 1)

In [16]:
def load_mnist_dataset():
    (train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

    # resize image to minimum dimensions for models
    # adding new pixels to sides
    train_images = np.expand_dims(train_images, axis=-1)
    train_images = tf.image.resize_with_crop_or_pad(train_images, target_height=32, target_width=32)

    test_images = np.expand_dims(test_images, axis=-1)
    test_images = tf.image.resize_with_crop_or_pad(test_images, target_height=32, target_width=32)

    train_images = tf.cast(train_images, tf.float32)
    test_images = tf.cast(test_images, tf.float32)

    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    return (train_images, train_labels), (test_images, test_labels)

In [17]:
(train_images, train_labels), (test_images, test_labels) = load_mnist_dataset()

### mobilenetV2

In [18]:
MODEL_PATH = './mnist_models/MobileNetV2_mnist_model.h5'

In [19]:
base_model = MobileNetV2(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./mnist_models/MobileNetV2_mnist_model.h5


### EfficientNets - B0, B4

#### EfficientNetB0

In [20]:
MODEL_PATH = './mnist_models/EfficentNetB0_mnist_model.h5'

In [21]:
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./mnist_models/EfficentNetB0_mnist_model.h5


#### EfficientNetB4

In [22]:
MODEL_PATH = './mnist_models/EfficentNetB4_mnist_model.h5'

In [23]:
base_model = EfficientNetB4(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./mnist_models/EfficentNetB4_mnist_model.h5


## Beans dataset

In [6]:
NUM_CLASSES = 3
INPUT_SHAPE = (500, 500, 3)
BATCH_SIZE = 32

In [7]:
def convert_ds_to_tensors(ds):
    """returns tuple of train_X, train_y"""
    a = ds.map(lambda a, b: a)
    tf_list = []
    for i in a:
        tf_list.append(i)
    train_X = tf.stack(tf_list, axis=0)
    
    b = ds.map(lambda a, b: b)
    tf_list = []
    for i in b:
        tf_list.append([i.numpy()])
    train_y = np.array(tf_list, dtype=np.uint8)
    return train_X, train_y

In [8]:
def load_beans_dataset():
    (ds_train, ds_test), ds_info = tfds.load(
        'beans',
        split=['train','test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    # convert to tensors and np array from tf.DataSet
    train_images, train_labels = convert_ds_to_tensors(ds_train)
    test_images, test_labels = convert_ds_to_tensors(ds_test)

    train_images = tf.cast(train_images, tf.float32)
    test_images = tf.cast(test_images, tf.float32)
    
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    return (train_images, train_labels), (test_images, test_labels)

In [9]:
(train_images, train_labels), (test_images, test_labels) = load_beans_dataset()

### mobilenetV2

In [10]:
MODEL_PATH = './beans_models/MobileNetV2_beans_model.h5'

In [11]:
base_model = MobileNetV2(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./beans_models/MobileNetV2_beans_model.h5


### EfficientNets - B0, B4

#### EfficientNetB0

In [12]:
MODEL_PATH = './beans_models/EfficentNetB0_beans_model.h5'

In [13]:
base_model = EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Model is already trained and saved here: ./beans_models/EfficentNetB0_beans_model.h5


#### EfficientNetB4

In [14]:
MODEL_PATH = './beans_models/EfficentNetB4_beans_model.h5'

In [ ]:
base_model = EfficientNetB4(include_top=False, weights=None, input_shape=INPUT_SHAPE)

model = wrap_model(base_model=base_model, num_classes=NUM_CLASSES)

train_model(model=model, train_X=train_images, train_y=train_labels, model_path=MODEL_PATH)

Epoch 1/2
